In [148]:
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import normalize

from sklearn.metrics.pairwise import euclidean_distances

from nltk.corpus import stopwords

In [149]:
# get data

# data - first 100k from https://drive.google.com/file/d/1c5-PBIwxmzudrR5JUJv_Q_-hq2Ea2CMh/edit

data = pd.read_json('rambler-news/100k.json', lines=True)[['title', 'descr']]
data.head()

,title,descr
0,В рязанской колонии заключенный ударил сотруд...,Следователи Скопинского МСО СУ СК РФ по Рязан...
1,«Работа такая…» Политолог о заявлениях на Укр...,История Российской Федерации закончится распа...
2,Роскосмос начал сбор анонимных жалоб на наруш...,"МОСКВА, 28 фев — РИА Новости. Госкорпорация «..."
3,Срочно: Возвращение культурных ценностей в КН...,"Вашингтон, 28 февраля /Синьхуа/ — Возвращение..."
4,Лекция о кондитерской династии Абрикосовых пр...,"МОСКВА, 28 февраля. /ТАСС/. Лекцию о знаменит..."


In [153]:
# tokenize data

vectorizer = CountVectorizer(
    min_df = 10, 
    max_df = 0.3,
    max_features=20000,
    stop_words=stopwords.words('russian'),
    token_pattern=r'[А-Я-а-яA-Za-z]+')

vectorizer.fit(np.concatenate([data['title'], data['descr']]))
vocab = {j:i for i, j in vectorizer.vocabulary_.items()}

titles = vectorizer.transform(data['title'])
descrs = vectorizer.transform(data['descr'])

In [154]:
data['title_tokens'] = [i.indices for i in titles]
data['descr_tokens'] = [i.indices for i in descrs]
data.head()

,title,descr,title_tokens,descr_tokens
0,В рязанской колонии заключенный ударил сотруд...,Следователи Скопинского МСО СУ СК РФ по Рязан...,"[3116, 4786, 6478, 7346, 15158, 16513, 18149]","[13, 967, 1104, 1621, 3092, 3093, 3116, 3610, ..."
1,«Работа такая…» Политолог о заявлениях на Укр...,История Российской Федерации закончится распа...,"[12017, 14184, 15009, 17350, 18247]","[190, 311, 364, 418, 523, 839, 1470, 1696, 194..."
2,Роскосмос начал сбор анонимных жалоб на наруш...,"МОСКВА, 28 фев — РИА Новости. Госкорпорация «...","[757, 4428, 8697, 8852, 13768, 14996, 15284, 1...","[0, 262, 326, 434, 757, 779, 1236, 1237, 1349,..."
3,Срочно: Возвращение культурных ценностей в КН...,"Вашингтон, 28 февраля /Синьхуа/ — Возвращение...","[2121, 6416, 7095, 14745, 16524, 16787, 16833,...","[1571, 2121, 2793, 5158, 6416, 7095, 7262, 733..."
4,Лекция о кондитерской династии Абрикосовых пр...,"МОСКВА, 28 февраля. /ТАСС/. Лекцию о знаменит...","[1608, 3840, 7226, 7625, 13805]","[8, 396, 867, 1608, 1640, 3085, 3171, 3242, 35..."


In [156]:
data = data[data['title_tokens'].map(lambda x: len(x)) != 0]
data = data[data['descr_tokens'].map(lambda x: len(x)) != 0]

In [188]:
embedding_size = 256
n_iters = 1000
batch_size = 1
lr = 0.01
total_loss = 0
print_every = 100

In [183]:
# initialize random word embeddings
w = np.random.normal(1e-6, 1e-7, size=(len(vocab), embedding_size))

In [184]:
# update weights
def update(w, vt, tokens):
    for token in tokens:
        w[token] -= vt
    return w

def loss(u, v, vv):
    # customized warp because X/N-1 is always const due to fact we sample data
    norm_uv = np.linalg.norm(u) * np.linalg.norm(v)
    norm_uvv = np.linalg.norm(u) * np.linalg.norm(vv)
    
    return np.dot(u,v) / norm_uv - np.dot(u,vv) / norm_uvv

In [258]:
gr_title = np.zeros(embedding_size)
gr_descr = np.zeros(embedding_size)
gr_neg = np.zeros(embedding_size)

for i in range(0, n_iters):
    for j in range(batch_size):
        sample = data.sample(n=2).values
        title = sample[0,2]
        descr = sample[0,3]
        neg = sample[1,3]
        
        # aggr
        title_w = np.max(w[title], axis=0)
        descr_w = np.max(w[descr], axis=0)
        neg_w = np.max(w[neg], axis=0)

        total_loss += loss(title_w, descr_w, neg_w)

        gr_title = gr_title + lr * (neg_w - descr_w)
        gr_descr = gr_descr - lr * title_w
        gr_neg = gr_neg + lr * title_w

        if (j + 1) % batch_size == 0:
            w = update(w, gr_title, title)
            w = update(w, gr_descr, descr)
            w = update(w, gr_neg, neg)
            
            # zero grad
            gr_title = np.zeros(embedding_size)
            gr_descr = np.zeros(embedding_size)
            gr_neg = np.zeros(embedding_size)
            w[w < 0] = 1e-16

    if i % print_every == 0:
        print('[{}/{} iterations. Loss - {}]'.format(i, n_iters, total_loss))
        total_loss = 0

[0/1000 iterations. Loss - 0.09474564252739137]
[100/1000 iterations. Loss - 0.15903216390828015]
[200/1000 iterations. Loss - 0.07207707129356045]
[300/1000 iterations. Loss - 0.2529516124329169]
[400/1000 iterations. Loss - 0.016311463557099504]
[500/1000 iterations. Loss - 0.16602353451279708]
[600/1000 iterations. Loss - 0.09080578263395689]
[700/1000 iterations. Loss - 0.1831070733380612]
[800/1000 iterations. Loss - 0.12292492872727212]
[900/1000 iterations. Loss - 0.18948962969800492]


In [260]:
# embedding distances

distances = euclidean_distances(w)
np.fill_diagonal(distances, np.inf)

In [193]:
# testing

[vocab[i] for i in np.argsort(distances[vectorizer.vocabulary_['трамп']])[:10]]

['штатов',
 'американский',
 'дональд',
 'трампа',
 'соединенных',
 'агентство',
 'вашингтоне',
 'конгресса',
 'переговоры',
 'военный']

In [197]:
# testing

[vocab[i] for i in np.argsort(distances[vectorizer.vocabulary_['путин']])[:10]]

['владимир',
 'совета',
 'стран',
 'частности',
 'безопасности',
 'председатель',
 'среди',
 'российская',
 'страны',
 'ситуации']

In [196]:
# testing

[vocab[i] for i in np.argsort(distances[vectorizer.vocabulary_['новости']])[:10]]

['москва',
 'мид',
 'будут',
 'российского',
 'подчеркнул',
 'тасс',
 'глава',
 'числе',
 'риа',
 'федерации']

In [204]:
# testing

[vocab[i] for i in np.argsort(distances[vectorizer.vocabulary_['украина']])[:10]]

['украинских',
 'петр',
 'киеве',
 'украиной',
 'провокации',
 'экономика',
 'президентских',
 'марта',
 'заявила',
 'ответ']

In [207]:
# testing

[vocab[i] for i in np.argsort(distances[vectorizer.vocabulary_['лавров']])[:10]]

['отношения',
 'отношений',
 'слова',
 'европы',
 'дипломат',
 'сфере',
 'законопроект',
 'договора',
 'национальной',
 'председателя']

In [208]:
# testing

[vocab[i] for i in np.argsort(distances[vectorizer.vocabulary_['telegram']])[:10]]

['западная',
 'протестующие',
 'питьевой',
 'семеро',
 'сайтах',
 'мюллера',
 'марио',
 'густой',
 'нигерии',
 'животного']

In [220]:
# testing

[vocab[i] for i in np.argsort(distances[vectorizer.vocabulary_['порошенко']])[:10]]

['киев',
 'украине',
 'властей',
 'киева',
 'требования',
 'считает',
 'сообщило',
 'лишь',
 'республики',
 'добавил']